In [ ]:
from IPython.core.debugger import set_trace

import numpy as np
import cv2
import glob
import pickle
from tracker import tracker
from utilities import *

dist_pickle = pickle.load(open('camera_cal/calibration.p','rb'))
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

In [ ]:
images = glob.glob('./test_images/test*.jpg')
format = './test_images/output/{1}{0}.jpg'

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    result, preprocess_image, warped, detected_windows = process_image(img=img, mtx=mtx,dist=dist, return_debug_images=True)   
    
    result_write_name = format.format('tracked',idx)
    cv2.imwrite(result_write_name,result)    
    
    preprocess_write_name =  format.format('preprocess', idx) 
    cv2.imwrite(preprocess_write_name, preprocess_image)
    
    cv2.imwrite(format.format('warped', idx), warped)
    
    cv2.imwrite(format.format('original', idx), img)
    
    cv2.imwrite(format.format('detected_windows', idx), detected_windows)
print('...')